In [0]:
dbutils.widgets.text('initial_load_flag','')
initial_load_flag = dbutils.widgets.get('initial_load_flag')

In [0]:
from pyspark.sql.functions import *
from pyspark.sql.types import *
from delta.tables import DeltaTable

In [0]:
# Key cols list
key_cols = "['flight_id']"
key_cols_list = eval(key_cols)

# catalog
catalog = 'flights'

#cdc column
cdc_column = 'modifiedDate'

#backdated refresh
backdated_refresh = ''

# source object
source_object = 'silver_flights'

#source schema
source_schema = 'silver'

# target object
target_object = 'flights'

#target schema
target_schema = 'gold'





In [0]:
# No back dated refresh
if len(backdated_refresh) == 0:

    #if table exists in the destination 
    if spark.catalog.tableExists(f'{catalog}.{target_schema}.{target_object}'):
        last_load = spark.sql(f"select max({cdc_column}) from flights.{target_schema}.{target_object}").collect()[0][0]
    else:

        last_load = '1900-01-01 00:00:00'
# yes back dated refresh
else:
    last_load = backdated_refresh
    


        

In [0]:

df = spark.sql(f'select * from {catalog}.{source_schema}.{source_object} where {cdc_column} >= "{last_load}" ')
df.display()

In [0]:
if spark.catalog.tableExists(f'{catalog}.{target_schema}.{target_object}'):
    df_old = spark.sql(f'select dimflightkey,flight_id,airline,destination,flight_date,create_date,update_date from {catalog}.{target_schema}.{target_object}')
else:
    df_old = spark.sql(f'select 0 dimflightkey,flight_id,airline,destination,flight_date,0 create_date,0 update_data from {catalog}.{source_schema}.{source_object} where 1=0')
df_old.display()


In [0]:
df_old = df_old.withColumnRenamed('dimflightkey','old_dimflightkey') \
        .withColumnRenamed('create_date','old_create_date') \
        .withColumnRenamed('update_data','old_update_date') \
        .withColumnRenamed('flight_id','old_flight_id') \
        .withColumnRenamed('airline','old_airline') \
        .withColumnRenamed('destination','old_destination') \
        .withColumnRenamed('flight_date','old_flight_date')
df_old.display()


In [0]:
df_join = df.join(df_old,df.flight_id == df_old.old_flight_id,'left')
df_join.display()




In [0]:
df_new = df_join.where(df_join.old_dimflightkey.isNull())
df_old = df_join.where(df_join.old_dimflightkey.isNotNull())
df_new.display()
df_old.display()


## Preparing df_old


In [0]:
df_old = df_old.drop('old_flight_id','old_update_date','old_airline','old_destination','old_flight_date')
df_old = df_old.withColumnRenamed('old_dimflightkey','dimflightkey')
df_old = df_old.withColumnRenamed('old_create_date','create_date')
df_old = df_old.withColumn('create_date',to_timestamp('create_date'))
df_old = df_old.withColumn('update_date',current_timestamp())




## Preparing df_new

In [0]:
df_new.display()

In [0]:
df_new = df_new.drop('old_flight_id','old_airline','old_destination','old_flight_date','old_dimflightkey','old_create_date','old_update_date')
df_new = df_new.withColumn('create_date',current_timestamp())
df_new = df_new.withColumn('update_date',current_timestamp())
df_new.display()

In [0]:
df_new = df_new.withColumn('dimflightkey',monotonically_increasing_id()+ lit(1))
df_new.display()


In [0]:
if (initial_load_flag =='0'):
    max_surrogate_key = 0
else:
    df_max_sur = spark.sql(f'select max(dimflightkey) as max_surrogate_key from {catalog}.{target_schema}.{target_object}')
    max_surrogate_key = df_max_sur.collect()[0]['max_surrogate_key']



In [0]:
df_new = df_new.withColumn('dimflightkey',lit(max_surrogate_key) + col('dimflightkey'))
df_new.display()

In [0]:
df_final = df_new.unionByName(df_old)


In [0]:
if spark.catalog.tableExists(f'{catalog}.{target_schema}.{target_object}'):
    dlt_obj = DeltaTable.forPath(spark,'abfss://flights@adls7428.dfs.core.windows.net/gold/flights')
    dlt_obj.alias("t").merge(
        df_final.alias("s"),
        "t.dimflightkey = s.dimflightkey") \
        .whenMatchedUpdateAll() \
        .whenNotMatchedInsertAll() \
        .execute()
else:
    df_final.write.mode('append').format('delta').option('path',f'abfss://flights@adls7428.dfs.core.windows.net/gold/{target_object}').saveAsTable(f'{catalog}.{target_schema}.{target_object}')